<center><h1>  1. zadaća </h1></center>
<center><h1>Algoritmi za određivanje konveksne ljuske skupa točaka </h1></center>

<h2> Konveksna ljuska </h2>

<h3> Uvod </h3>

Za početak ćemo definirati osnovne pojmove vezane uz temu ovoga rada. Kažemo da je skup $ D \subseteq \mathbb{R}^{n} $ konveksan ako za bilo koje dvije točke $ x_{1}, x_{2} \in \mathbb{R}^{n} $ sadrži i segment određen tim točkama.

$$ x_{1}, x_{2} \in D \Rightarrow \lambda x_{1} + ( 1 - \lambda ) x_{2}  \in D , \:\: \forall \lambda \in [ 0, 1 ]$$


<h3> Vrste algoritama </h3> 
- zajedničke stvari 
- baratanje točkama - klasa
- udaljenosti, kutevi, "lijeve baze"

In [5]:
# izrada klase Točka
class Tocka:
    def __init__(self, x_, y_):
        self.x = x_
        self.y = y_
    
import random as rand

# generiranje skupa od 100 random točaka, upisati ih u neki file?
lista_tocaka = []
for x in range(100):
    x = rand.randint(1,100)
    y = rand.randint(1,100)
    tocka = Tocka(x,y)
    lista_tocaka.append(tocka)

# ispis točaka
#for x in range(100):
#    print(lista_tocaka[x].x,", ",lista_tocaka[x].y)

In [6]:
# funkcija lijeva_baza provjerava u kojem smjeru se tocka C nastavlja na segment AB
# vraca pozitivan rezultat ukoliko se C nalazi lijevo gledajuci od A prema B pa k C
# vraca negativan rezultat ukoliko se C nalazi desno gledajuci od A prema B pa k C
# te vraca nulu ukoliko su te tri tocke kolinearne
def lijeva_baza( tocka_A, tocka_B, tocka_C):
    return (tocka_A.x * (tocka_B.y - tocka_C.y)) 
            + tocka_B.x * (tocka_C.y - tocka_A.y) 
            + tocka_C.x * (tocka_A.y - tocka_B.y);
#provjeriti u pravilima, kako se ovi dugi izrazi pravilno zapisuju

In [12]:
X = Tocka(1, 1)
Y = Tocka(1, 2)
Z = Tocka(1, 3)

print( lijeva_baza(X,Y,Z) )
Z = Tocka(3,3)

print( lijeva_baza(X,Y,Z))

Z = Tocka(0, 7)
print( lijeva_baza(X,Y,Z))

0
-2
1


In [13]:
# kako nam trebaju samo omjeri udaljenosti, a ne same vrijednosti udaljenosti možemo računati kvadrat,
# te izostaviti računanje korijena
def udaljenost_tocaka( tocka_A, tocka_B):
    return (tocka_A.x - tocka_B.x)*(tocka_A.x - tocka_B.x) + (tocka_A.y - tocka_B.y)*(tocka_A.y- tocka_B.y);

In [17]:
print( udaljenost_tocaka(X,Y) > udaljenost_tocaka(Y,Z))
print( udaljenost_tocaka(X,Y) >= udaljenost_tocaka(Y,X))

False
True


In [27]:
# kao početnu točku tražim najnižu točku, dakle onu s najmanjom y vrijednosti
# ukoliko dvije točke imaju jednaku najmanju y vrijednost, tražimo onu "lijeviju", dakle s manjom x vrijednosti
def Y_min_tocka( lista_tocaka):
    tocka_minY = lista_tocaka[0]
    for tocka in lista_tocaka:
        if ( ( tocka.y < tocka_minY.y ) or ( tocka.y == tocka_minY.y and tocka.x < tocka_minY.x ) ):
            tocka_minY = tocka
    return tocka_minY    

In [28]:
lista_tocaka2 = []
pocetna_tocka = Tocka(0,0)
for x in range(5):
    x = rand.randint(1,10)
    y = rand.randint(1,10)
    tocka = Tocka(x,y)
    lista_tocaka2.append(tocka)
    print("X : ",x,", Y :",y)
pocetna_tocka = Y_min_tocka(lista_tocaka2)

print(pocetna_tocka.x, pocetna_tocka.y)

X :  7 , Y : 2
X :  5 , Y : 8
X :  3 , Y : 10
X :  2 , Y : 6
X :  4 , Y : 8
7 2


<h3> Graham algoritam </h3>

- opcenito o algoritmu
- pseudokod
- implementacija ( dio po dio )
- graf
- animacija?

<h3> Jarvis march </h3>

- opcenito o algoritmu
- pseudokod
- implementacija ( dio po dio )
- graf
- animacija?

<h3> Primjena </h3>

- nabrojati primjere
- implementacija ( dio po dio )

<h3> 3D konveksne ljuske ( konveksna zatvorenja ) </h3>
- nagovještaj 